### check valid (return > 0)

In [3]:
import os
import re

def process_tpl_files(directory):
    unique_prefixes = set()
    pattern = re.compile(r'query_condition_\d+\.tpl$')
    
    for root, _, files in os.walk(directory):
        for file in files:
            if pattern.match(file):
                file_path = os.path.join(root, file)
                
                with open(file_path, 'r') as f:
                    content = f.read()
                    lines = content.split('\n')
                    
                    # remove define stmt
                    non_define_lines = [line.strip() for line in lines 
                                      if line.strip() and not line.strip().startswith('define')]
                    
                    for line in non_define_lines:
                        unique_prefixes.add(line.split('_')[0] + '_')
    print("Found unique prefixes:")
    for prefix in sorted(unique_prefixes):
        print(prefix)
    
    return unique_prefixes

directory = "raw_tpl_files"
# prefixes = process_tpl_files(directory)

Found unique prefixes:
;_
c_
ca_
cc_
cd1.cd_
cd_
cr_
cs_
d1.d_
d2.d_
d3.d_
d_
hd_
i_
ib_
item1.i_
item2.i_
s1.ss_
s2.ss_
s_
sm_
ss_
w_
ws_


In [8]:
import os
import glob
import json

# Get all .dat files and extract table names
tables = sorted([os.path.basename(f)[:-4] for f in glob.glob('/home/lsh/test_kepler/datasets/tpcds/*.dat')])

# Create dictionary with concatenated prefixes as values
table_dict = {table: ''.join(word[0] for word in table.split('_')) + '_' for table in tables}

# with open('table_dict.json', 'w') as f:
#     json.dump(table_dict, f)
#     print(table_dict)

{'call_center': 'cc_', 'catalog_page': 'cp_', 'catalog_returns': 'cr_', 'catalog_sales': 'cs_', 'customer': 'c_', 'customer_address': 'ca_', 'customer_demographics': 'cd_', 'date_dim': 'dd_', 'dbgen_version': 'dv_', 'household_demographics': 'hd_', 'income_band': 'ib_', 'inventory': 'i_', 'item': 'i_', 'promotion': 'p_', 'reason': 'r_', 'ship_mode': 'sm_', 'store': 's_', 'store_returns': 'sr_', 'store_sales': 'ss_', 'time_dim': 'td_', 'warehouse': 'w_', 'web_page': 'wp_', 'web_returns': 'wr_', 'web_sales': 'ws_', 'web_site': 'ws_'}


In [17]:
import re
import json
from pathlib import Path

def extract_tables_from_query(file_path):
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        
        tables_str = content.lower().split('select')[1].split('from')[1].split('where')[0]
        tables = [
            table.strip() 
            for table in tables_str.split(',')
            if table.strip()
        ]
        
        return tables
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return []

def process_all_queries(directory_path):
    result = {}
    path = Path(directory_path)
    
    for file_path in path.glob("query*_spj.tpl"):
        try:
            query_id = re.search(r'query(\d+)_spj\.tpl', file_path.name)
            if query_id:
                query_id = query_id.group(1)
                # print(query_id)
                tables = extract_tables_from_query(str(file_path))
                if tables:
                    result[query_id] = tables
        except Exception as e:
            print(f"ERROR {file_path}: {str(e)}")
            continue
    
    return result

# if __name__ == "__main__":
#     tpl_directory = "raw_tpl_files"
#     result_dict = process_all_queries(tpl_directory)
    
#     sorted_dict = dict(sorted(result_dict.items(), key=lambda x: int(x[0])))
    
#     with open('query_table.json', 'w') as f:
#         json.dump(sorted_dict, f, indent=4)
#     print(sorted_dict)

{'013': ['store_sales', 'store', 'customer_demographics', 'household_demographics', 'customer_address', 'date_dim'], '018': ['catalog_sales', 'customer_demographics', 'customer', 'customer_address', 'date_dim', 'item'], '019': ['date_dim', 'store_sales', 'item', 'customer', 'customer_address', 'store'], '025': ['store_sales', 'store_returns', 'catalog_sales', 'date_dim d1', 'date_dim d2', 'date_dim d3', 'store', 'item'], '027': ['store_sales', 'customer_demographics', 'date_dim', 'store', 'item'], '040': ['catalog_sales\nleft outer join catalog_returns on (cs_order_number = cr_order_number\n                                    and cs_item_sk = cr_item_sk)', 'warehouse', 'item', 'date_dim'], '050': ['store_sales', 'store_returns', 'store', 'date_dim d1', 'date_dim d2'], '072': ['catalog_sales\njoin inventory on (cs_item_sk = inv_item_sk)\njoin warehouse on (w_warehouse_sk=inv_warehouse_sk)\njoin item on (i_item_sk = cs_item_sk)\njoin customer_demographics on (cs_bill_cdemo_sk = cd_demo_s

### generate PQO files

In [22]:
import pandas as pd

df = pd.read_csv('gaussian_valid_251.csv')
df['query_id'] = df['query_id'].astype(str).str.zfill(3)
result_dict = df.groupby('query_id')['query_instance'].agg(list).to_dict()

print(f"Length of query_id: {len(result_dict)}")
print("Key:", sorted(result_dict.keys()))
print("Instance length:", set([len(value) for key, value in sorted(result_dict.items(), key=lambda x: x[0])]))
print(result_dict)

Length of query_id: 13
Key: ['013', '018', '019', '025', '027', '040', '050', '072', '084', '085', '091', '099', '100']
Instance length: {251}
{'013': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 176, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 1

In [25]:
import pandas as pd
import numpy as np
import json
from typing import Dict, List

np.random.seed(2024)

# split train & test
train_test_dict = {}

# shuffle instance list
for query_id, instances in result_dict.items():
    instances = np.array(instances)
    np.random.shuffle(instances)
    instances = instances.tolist()
    
    # 50 training + 200 testing
    train_instances = instances[:50]
    test_instances = instances[50:250]
    
    # save result
    train_test_dict[query_id] = {
        "train": train_instances,
        "test": test_instances
    }

with open('0_train_test.json', 'w') as f:
    json.dump(train_test_dict, f, indent=2)

print(f"all query length: {len(train_test_dict)}")
print(f"train length: {set([len(value['train']) for key, value in sorted(train_test_dict.items(), key=lambda x: x[0])])}")
print(f"test length: {set([len(value['test']) for key, value in sorted(train_test_dict.items(), key=lambda x: x[0])])}")


all query length: 13
train length: {50}
test length: {200}


### without table alias

In [40]:
import json
import os
from typing import Dict, List
from collections import defaultdict

def load_train_test_data(json_file: str = '0_train_test.json') -> Dict:
    with open(json_file, 'r') as f:
        return json.load(f)

def load_table_dict(json_file: str = '0_table_dict.json') -> Dict:
    with open(json_file, 'r') as f:
        return json.load(f)

def read_sql_file(file_path: str) -> str:
    with open(file_path, 'r') as f:
        return f.read()

def process_condition_files(query_id: str, instance_list: List[str], mode: str, table_dict: Dict) -> Dict[str, List[str]]:
    table_conditions = defaultdict(list)
    
    for i in instance_list:
        current_instance_conditions = defaultdict(list)
        
        condition_file = f"queries/query_condition_{query_id}/query_condition_{query_id}_{i}.sql"
        content = read_sql_file(condition_file)
        
        for line in content.strip().split('\n'):
            if line.strip():
                table_key = line.strip().split('_')[0]
                table_name = table_dict.get(table_key)
                if table_name:
                    current_instance_conditions[table_name].append(line.strip())
        
        for table_name, conditions in current_instance_conditions.items():
            instance_str = '["' + '", "'.join(conditions) + '"]'
            table_conditions[table_name].append(instance_str)
    
    for table_name, instance_arrays in table_conditions.items():
        output_file = f"PQO/{query_id}/{query_id}_{len(instance_list)}_{mode}_{table_name}.txt"
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        
        with open(output_file, 'w') as f:
            f.write(',\n'.join(instance_arrays))
    
    return table_conditions

def process_query_files(query_id: str, instance_list: List[str], mode: str):
    queries_dict = {}
    
    for idx, i in enumerate(instance_list):
        query_file = f"queries/query{query_id}_spj/query{query_id}_spj_{i}.sql"
        content = read_sql_file(query_file)
        
        key = f"{query_id}_{mode}_{len(instance_list)}_{idx}"
        queries_dict[key] = content
    
    # create json file for each query
    output_file = f"PQO/{query_id}/{query_id}_{mode}_{len(instance_list)}.json"
    with open(output_file, 'w') as f:
        json.dump(queries_dict, f, indent=2)
    
    return queries_dict

def main():
    train_test_data = load_train_test_data()
    table_dict = load_table_dict()
    query_id = '099'
    print(f"Handling Query {query_id}")

    train_instances = train_test_data[query_id]['train']
    test_instances = train_test_data[query_id]['test']

    print(f"Training ({len(train_instances)})")
    process_condition_files(query_id, train_instances, 'train', table_dict)
    process_query_files(query_id, train_instances, 'training')
    
    print(f"Testing ({len(test_instances)})...")
    process_condition_files(query_id, test_instances, 'test', table_dict)
    process_query_files(query_id, test_instances, 'testing')

if __name__ == "__main__":
    # main()

Handling Query 099
Training (50)
Testing (200)...


### with table alias: 025, 050, 072, 085, 100

In [ ]:
import json
import os
from typing import Dict, List
from collections import defaultdict

def load_train_test_data(json_file: str = '0_train_test.json') -> Dict:
    with open(json_file, 'r') as f:
        return json.load(f)

def load_table_dict(json_file: str = '0_table_dict.json') -> Dict:
    with open(json_file, 'r') as f:
        return json.load(f)

def read_sql_file(file_path: str) -> str:
    with open(file_path, 'r') as f:
        return f.read()

def process_condition_files(query_id: str, instance_list: List[str], mode: str, table_dict: Dict) -> Dict[str, List[str]]:
    table_conditions = defaultdict(list)
    
    for i in instance_list:
        current_instance_conditions = defaultdict(list)
        
        condition_file = f"queries/query_condition_{query_id}/query_condition_{query_id}_{i}.sql"
        content = read_sql_file(condition_file)
        
        for line in content.strip().split('\n'):
            if line.strip():
                parts = line.strip().split('_')[0]
    
                # check table alias
                if '.' in parts:
                    # e.g. d1.d_moy
                    table_alias_number = parts.split('.')[0][-1]  # get 1
                    table_key = parts.split('.')[1]  # get d
                else:
                    # original case
                    table_key = parts
                    table_alias_number = None
                
                table_name = table_dict.get(table_key)
                if table_name:
                    if table_alias_number is not None:
                        current_instance_conditions[f"{table_name}_{table_alias_number}"].append(line.strip())
                    else:
                        current_instance_conditions[table_name].append(line.strip())
        
        for table_name, conditions in current_instance_conditions.items():
            instance_str = '["' + '", "'.join(conditions) + '"]'
            table_conditions[table_name].append(instance_str)
    
    for table_name, instance_arrays in table_conditions.items():
        output_file = f"PQO/{query_id}/{query_id}_{len(instance_list)}_{mode}_{table_name}.txt"
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        
        with open(output_file, 'w') as f:
            f.write(',\n'.join(instance_arrays))
    
    return table_conditions

def process_query_files(query_id: str, instance_list: List[str], mode: str):
    queries_dict = {}
    
    for idx, i in enumerate(instance_list):
        query_file = f"queries/query{query_id}_spj/query{query_id}_spj_{i}.sql"
        content = read_sql_file(query_file)
        
        key = f"{query_id}_{mode}_{len(instance_list)}_{idx}"
        queries_dict[key] = content
    
    # create json file for each query
    output_file = f"PQO/{query_id}/{query_id}_{mode}_{len(instance_list)}.json"
    with open(output_file, 'w') as f:
        json.dump(queries_dict, f, indent=2)
    
    return queries_dict

def main():
    train_test_data = load_train_test_data()
    table_dict = load_table_dict()
    
    for query_id in ['025', '050', '072', '085', '100']:
        print(f"Handling Query {query_id}")

        train_instances = train_test_data[query_id]['train']
        test_instances = train_test_data[query_id]['test']

        print(f"Training ({len(train_instances)})")
        process_condition_files(query_id, train_instances, 'train', table_dict)
        process_query_files(query_id, train_instances, 'training')
        
        print(f"Testing ({len(test_instances)})...")
        process_condition_files(query_id, test_instances, 'test', table_dict)
        process_query_files(query_id, test_instances, 'testing')

if __name__ == "__main__":
    # main()

Handling Query 025
Training (50)
Testing (200)...
Handling Query 050
Training (50)
Testing (200)...
Handling Query 072
Training (50)
Testing (200)...
Handling Query 085
Training (50)
Testing (200)...
Handling Query 100
Training (50)
Testing (200)...


### generate query_id: param list

In [1]:
import json
import os

def read_sql_file(file_path):
    """
    Read SQL file and process its content according to requirements.
    Returns a list of statements after processing.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Split by newlines and process each line
        stmts = []
        for line in content.split('\n'):
            line = line.strip()
            # Skip single characters
            if len(line) <= 1:
                continue
            stmts.append(line)
        
        return stmts
    except Exception as e:
        print(f"Error reading SQL file {file_path}: {str(e)}")
        return []

def process_files():
    # Read the input JSON file
    try:
        with open('0_train_test.json', 'r', encoding='utf-8') as f:
            train_test_data = json.load(f)
    except Exception as e:
        print(f"Error reading input JSON file: {str(e)}")
        return

    # Output dictionary
    output_data = {}

    # Process each query
    for query_id, data in train_test_data.items():
        output_data[query_id] = {"train": [], "test": []}
        
        # Process train instances
        for idx, i in enumerate(data.get("train", [])):
            sql_file = f"queries/query_condition_{query_id}/query_condition_{query_id}_{i}.sql"
            if os.path.exists(sql_file):
                stmts = read_sql_file(sql_file)
                output_data[query_id]["train"].append(stmts)
            else:
                print(f"Warning: Train SQL file not found: {sql_file}")
        
        # Process test instances
        for idx, i in enumerate(data.get("test", [])):
            sql_file = f"queries/query_condition_{query_id}/query_condition_{query_id}_{i}.sql"
            if os.path.exists(sql_file):
                stmts = read_sql_file(sql_file)
                output_data[query_id]["test"].append(stmts)
            else:
                print(f"Warning: Test SQL file not found: {sql_file}")

    # Save the output JSON file
    try:
        with open('PQO_gaussian_sql.json', 'w', encoding='utf-8') as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2)
        print("Successfully processed files and saved output to PQO_gaussian_sql.json")
    except Exception as e:
        print(f"Error saving output JSON file: {str(e)}")

if __name__ == "__main__":
    process_files()

Successfully processed files and saved output to PQO_gaussian_sql.json


In [2]:
import json
from collections import defaultdict

def analyze_json_structure():
    # Read the JSON file
    try:
        with open('PQO_gaussian_sql.json', 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error reading JSON file: {str(e)}")
        return

    # Analyze each query
    for query_id, content in data.items():
        print(f"\nQuery {query_id}:")
        
        # Analyze train data
        train_data = content.get('train', [])
        train_len = len(train_data)
        print(f"Train list length: {train_len}")
        
        # Check train instance lengths
        train_instance_lengths = [len(instance) for instance in train_data]
        train_lengths_consistent = len(set(train_instance_lengths)) == 1
        
        if train_lengths_consistent:
            print(f"Train instances have consistent length: {train_instance_lengths[0]}")
        else:
            print("Train instances have varying lengths:")
            length_count = defaultdict(int)
            for length in train_instance_lengths:
                length_count[length] += 1
            for length, count in sorted(length_count.items()):
                print(f"  Length {length}: {count} instances")
        
        # Analyze test data
        test_data = content.get('test', [])
        test_len = len(test_data)
        print(f"Test list length: {test_len}")
        
        # Check test instance lengths
        test_instance_lengths = [len(instance) for instance in test_data]
        test_lengths_consistent = len(set(test_instance_lengths)) == 1
        
        if test_lengths_consistent:
            print(f"Test instances have consistent length: {test_instance_lengths[0]}")
        else:
            print("Test instances have varying lengths:")
            length_count = defaultdict(int)
            for length in test_instance_lengths:
                length_count[length] += 1
            for length, count in sorted(length_count.items()):
                print(f"  Length {length}: {count} instances")

if __name__ == "__main__":
    analyze_json_structure()


Query 013:
Train list length: 50
Train instances have consistent length: 6
Test list length: 200
Test instances have consistent length: 6

Query 018:
Train list length: 50
Train instances have consistent length: 6
Test list length: 200
Test instances have consistent length: 6

Query 019:
Train list length: 50
Train instances have consistent length: 5
Test list length: 200
Test instances have consistent length: 5

Query 025:
Train list length: 50
Train instances have consistent length: 3
Test list length: 200
Test instances have consistent length: 3

Query 027:
Train list length: 50
Train instances have consistent length: 4
Test list length: 200
Test instances have consistent length: 4

Query 040:
Train list length: 50
Train instances have consistent length: 4
Test list length: 200
Test instances have consistent length: 4

Query 050:
Train list length: 50
Train instances have consistent length: 3
Test list length: 200
Test instances have consistent length: 3

Query 072:
Train list leng